#### Usa o catalog do Setor Farmacêutico

In [0]:
%sql
USE catalog setorfarmaceutico;

In [0]:
%sql
DESCRIBE DETAIL setorfarmaceutico.bronze.principios_ativos;

##### Cria a tabela "principios_ativos" usando pySpark

In [0]:
df = spark.table("setorfarmaceutico.bronze.principios_ativos")
df.show()

##### Descreve o tipo de dado de cada coluna

In [0]:
df.printSchema()

##### **Garantia dos tipos de dados corretos**

In [0]:
from pyspark.sql.functions import col, mean

df = df.withColumn("nome_principio_ativo", col("nome_principio_ativo").cast("string")) \
       .withColumn("status", col("status").cast("string")) \
       .withColumn("quantidade_disponível", col("quantidade_disponível").cast("int")) \
       .withColumn("fabricante", col("fabricante").cast("string")) \
       .withColumn("valor_unitário", col("valor_unitário").cast("float")) \
       .withColumn("local_armazenamento", col("local_armazenamento").cast("string"))

In [0]:
from pyspark.sql.functions import mean, col, round, format_number

estoque_medio_status = (
  df.groupBy("status")
    .agg(mean(col("quantidade_disponível")).alias("estoque_medio"))
    .withColumn("estoque_medio", format_number(col("estoque_medio"), 2))
    .orderBy(col("estoque_medio").desc())
)

estoque_medio_status.show()


##### O estoque médio é quase igual entre itens disponíveis e indisponíveis, o que indica:
- A indisponibilidade não está ligada ao estoque baixo.
- Pode ser problema de logística, vencimento, compliance etc.

In [0]:
top_estoque = (
    df.orderBy(col("quantidade_disponível").desc())
      .select("nome_principio_ativo", "quantidade_disponível")
      .limit(10)
)

top_estoque.show()

##### Medicamentos de alta demanda, amplamente utilizados.

In [0]:
from pyspark.sql.functions import mean, col, round

valor_unit_medio_fabricante = (
    df.groupBy("fabricante")
      # .agg(mean(col("valor_unitário")).alias("valor_unit_medio"))
      .agg(round(mean(col("valor_unitário")), 2).alias("valor_unit_medio"))
      .orderBy(col("valor_unit_medio").desc())
      .limit(10)
)

valor_unit_medio_fabricante.show()

##### Pequenas variações → mercado competitivo. / Pode se usado para:
- Negociação
- Comparação de Margens
- Análise de custo de oportunidade

In [0]:
from pyspark.sql.functions import col, format_number

status_counts = (
    df.groupBy("status")
      .count()
      .orderBy(col("count").desc())
      .withColumn("count", format_number(col("count"), 2))
)

status_counts.show()

##### Muito balanceado: praticamente metade disponível e metade indisponível.

#### Criação da tabela no schema Silver após a EDA (Análise exploratória dos Dados)

In [0]:
%sql
use schema silver;

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.stg_principios_ativos")


### **RESUMO**

- Dataset contém 100 mil linhas, 6 atributos.

- Status dos itens é equilibrado (50%/50%).

- 4 fabricantes dominam igualmente o fornecimento.

- Estoque médio não diferencia disponível vs indisponível → indica problema operacional, não de produção.

- Principais princípios ativos com maior estoque são ligados a medicamentos de alta demanda (Metformina, Ibuprofeno, Paracetamol etc.).

- Preço médio por fabricante é muito semelhante → área com oportunidade de redução via negociação.